In [1]:
import pandas as pd
import mibian as mb
import yfinance as yf
from datetime import datetime

/Users/jteruya/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Black Scholes Assumptions

In [2]:
## Assume volatility is 20
vol = 20

## Assume free interest rate is 3%
interest_rate=3

## Yahoo Finance Ticker Object

In [3]:
stock_ticker = "CCL"
ticker = yf.Ticker(stock_ticker)

## Input DataFrame Helper Function

In [4]:
# Helper Functions for Constant Fields
def _constant_fields(df, expiration_date, interest_rate):
    df['underlying_price'] = ticker.history(period="0d").iloc[0]['Close']
    df['interest_rate'] = interest_rate
    df['days_to_expiration'] = (datetime.strptime(expiration_date, '%Y-%m-%d') - datetime.now()).days + 1

# Generate Mibian Input DF for Call Chains
def call_chain_input(ticker, expiration_date, interest_rate):
    df = pd.DataFrame(ticker.option_chain(expiration_date).calls[['strike']])
    _constant_fields(df, expiration_date, interest_rate)
    return df[['underlying_price', 'strike', 'interest_rate', 'days_to_expiration']]

# Generate Mibian Input DF for Put Chains
def put_chain_input(ticker, expiration_date, interest_rate):
    df = pd.DataFrame(ticker.option_chain(expiration_date).puts[['strike']])
    _constant_fields(df, expiration_date, interest_rate)
    return df[['underlying_price', 'strike', 'interest_rate', 'days_to_expiration']]

## Generate Black Scholes Input DataFrame Across Multiple Options Expirations
This steps takes a little time...

In [5]:
# Calls DF
call_input_df = pd.DataFrame()
# Puts DF
put_input_df = pd.DataFrame()

for expiration_date in ticker.options:
    call_input_df = call_input_df.append(call_chain_input(ticker=ticker, expiration_date=expiration_date, interest_rate=interest_rate), ignore_index=True)
    put_input_df = put_input_df.append(put_chain_input(ticker=ticker, expiration_date=expiration_date, interest_rate=interest_rate), ignore_index=True)

## Create Deep Copy of Input DataFrame
The previous step takes a while so this allows for rerunnability without having to re-run the previous step.

In [6]:
call_df = call_input_df.copy(deep=True)
put_df = put_input_df.copy(deep=True)

## Black Scholes Helper Functions

In [7]:
def black_scholes_vol(row, vol):
    return mb.BS(row.tolist(), volatility=vol)

def black_scholes_call_output(mb_object):
    return mb_object.callTheta, mb_object.callRho, mb_object.vega, mb_object.gamma

def black_scholes_put_output(mb_object):
    return mb_object.callTheta, mb_object.callRho, mb_object.vega, mb_object.gamma

## Generate Black Scholes Objects and Values

In [8]:
call_df['black_scholes'] = call_df.apply(lambda x: black_scholes_vol(x, vol), axis=1)
call_df['ticker'] = stock_ticker
call_df['black_scholes_call_theta'], call_df['black_scholes_call_rho'], call_df['black_scholes_vega'], call_df['black_scholes_gamma'] = \
    zip(*call_df['black_scholes'].map(lambda x: black_scholes_call_output(x)))

put_df['black_scholes'] = put_df.apply(lambda x: black_scholes_vol(x, vol), axis=1)
put_df['ticker'] = stock_ticker
put_df['black_scholes_put_theta'], put_df['black_scholes_put_rho'], put_df['black_scholes_vega'], put_df['black_scholes_gamma'] = \
    zip(*put_df['black_scholes'].map(lambda x: black_scholes_put_output(x)))

## Preview Call Output

In [9]:
call_df.head(10)

,underlying_price,strike,interest_rate,days_to_expiration,black_scholes,ticker,black_scholes_call_theta,black_scholes_call_rho,black_scholes_vega,black_scholes_gamma
0,17.83,1.0,3,5,<mibian.BS object at 0x11bd90bb0>,CCL,-0.000082,0.000137,0.000000e+00,0.000000e+00
1,17.83,2.0,3,5,<mibian.BS object at 0x11bd90040>,CCL,-0.000164,0.000274,0.000000e+00,0.000000e+00
2,17.83,8.0,3,5,<mibian.BS object at 0x11bd90280>,CCL,-0.000657,0.001095,8.764194e-258,1.006241e-255
3,17.83,9.0,3,5,<mibian.BS object at 0x11bd902b0>,CCL,-0.000739,0.001232,2.118979e-188,2.432859e-186
4,17.83,9.5,3,5,<mibian.BS object at 0x11bd90f70>,CCL,-0.000781,0.001301,3.117430e-160,3.579208e-158
5,17.83,10.0,3,5,<mibian.BS object at 0x11bd903d0>,CCL,-0.000822,0.001369,1.187503e-135,1.363406e-133
6,17.83,10.5,3,5,<mibian.BS object at 0x11bd909d0>,CCL,-0.000863,0.001438,3.317283e-114,3.808665e-112
7,17.83,11.0,3,5,<mibian.BS object at 0x11bd903a0>,CCL,-0.000904,0.001506,1.629172e-95,1.870498e-93
8,17.83,11.5,3,5,<mibian.BS object at 0x11bd90670>,CCL,-0.000945,0.001575,2.948475e-79,3.385226e-77
9,17.83,12.0,3,5,<mibian.BS object at 0x11bd90310>,CCL,-0.000986,0.001643,3.695364e-65,4.242749e-63


## Preview Put Output

In [10]:
put_df.head(10)

,underlying_price,strike,interest_rate,days_to_expiration,black_scholes,ticker,black_scholes_put_theta,black_scholes_put_rho,black_scholes_vega,black_scholes_gamma
0,17.83,5.0,3,5,<mibian.BS object at 0x11bd90430>,CCL,-0.000411,0.000685,0.000000e+00,0.000000e+00
1,17.83,6.0,3,5,<mibian.BS object at 0x11bd90460>,CCL,-0.000493,0.000822,0.000000e+00,0.000000e+00
2,17.83,7.0,3,5,<mibian.BS object at 0x11bdd5d00>,CCL,-0.000575,0.000959,0.000000e+00,0.000000e+00
3,17.83,8.0,3,5,<mibian.BS object at 0x11bdd5040>,CCL,-0.000657,0.001095,8.764194e-258,1.006241e-255
4,17.83,8.5,3,5,<mibian.BS object at 0x11bdd5ac0>,CCL,-0.000698,0.001164,1.068053e-220,1.226261e-218
5,17.83,9.0,3,5,<mibian.BS object at 0x11bdd5d60>,CCL,-0.000739,0.001232,2.118979e-188,2.432859e-186
6,17.83,9.5,3,5,<mibian.BS object at 0x11bdd5d90>,CCL,-0.000781,0.001301,3.117430e-160,3.579208e-158
7,17.83,10.0,3,5,<mibian.BS object at 0x11bdd5dc0>,CCL,-0.000822,0.001369,1.187503e-135,1.363406e-133
8,17.83,10.5,3,5,<mibian.BS object at 0x11bdd5df0>,CCL,-0.000863,0.001438,3.317283e-114,3.808665e-112
9,17.83,11.0,3,5,<mibian.BS object at 0x11bdd5e20>,CCL,-0.000904,0.001506,1.629172e-95,1.870498e-93
